In [1]:
# all the imports used throughout the notebook:
import json
import pandas as pd
import numpy as np
import wrangle
import acquire
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords, prep_string_data#, split_data
import scipy as sp
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

import nltk

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define the df
df = wrangle.get_npr_data()

# sample down due to size issuees
small_df = df.sample(100_000, random_state=222)
# counts of observations per speaker
counts = small_df['speaker'].value_counts()
# limiting our df to only speakers with 3 or more utterances. This helped wheen stratifying the splits
rest = small_df[~small_df['speaker'].isin(counts[counts < 3].index)]

The df has 1889857 rows and 12 columns.


In [3]:
# get initial splits based no "rest", which is a sampled-down df with speakers with 2 or less observations removed
train, validate, test = wrangle.split_data(rest)
print(train.shape)
print(validate.shape)
print(test.shape)

(42327, 11)
(18141, 11)
(15118, 11)


In [4]:
train.head()

,story_id_num,utterance_order,speaker,utterance,program,title,is_host,clean,lemmatized,vader,date
2635561,50926,18,leila fadel byline,"In the next scene, a woman scolds Hassan, assu...",morning edition,muslims are having a hollywood moment,False,"in the next scene , a woman scolds hassan , as...","in the next scene , a woman scold hassan , ass...",-0.4019,2018-10-30
2827216,9080,12,david french,"So for example, you - there is a - one kind of...",weekend edition saturday,praying in response to mass shootings,False,"so for example , you there is a one kind of so...","so for example , you there is a one kind of so...",-0.8856,2017-11-11
1751752,122898,5,melissa block,�as in to remove someone as a friend on a soci...,all things considered,dictionary picks 'unfriend' as word of the year,True,as in to remove someone as a friend on a socia...,a in to remove someone a a friend on a social ...,0.4939,2009-11-17
1875074,76606,69,jeff,Thank you.,talk of the nation,"super bowl ads: the good, the bad...",False,thank you .,thank you .,0.3612,2007-02-05
451635,64728,189,neal conan,Here's finally an e-mail from Carrie(ph) in La...,talk of the nation,the raw and emotional 'secret lives of boys',True,here s finally an e mail from carrieph in lafa...,here s finally an e mail from carrieph in lafa...,0.4329,2009-05-04


In [5]:
# define the features to go into the model
features = train.drop(columns = ['story_id_num','speaker','utterance','program','title',\
    'is_host','clean','lemmatized']).columns.to_list()

In [6]:
# sample down due to size issuees
small_df = df.sample(100_000, random_state=222)
# counts of observations per speaker
counts = small_df['speaker'].value_counts()
# limiting our df to only speakers with 3 or more utterances. This helped wheen stratifying the splits
rest = small_df[~small_df['speaker'].isin(counts[counts < 3].index)]

In [7]:
train, validate, test = wrangle.split_data(rest)
print(train.shape)
print(validate.shape)
print(test.shape)

(42327, 11)
(18141, 11)
(15118, 11)


In [8]:
# X_ and y_ splits


X_train = train[features]
y_train = train.is_host


X_validate = validate[features]
y_validate = validate.is_host


X_test = test[features]
y_test = test.is_host

In [9]:
# add a column to each basic split for the "actual" result (i.e. y_)
train['actual'] = y_train
validate['actual'] = y_validate
test['actual'] = y_test

In [10]:
# Create the empty dataframe
eval_df = pd.DataFrame(columns=['Model_Type', 'Train_Accuracy','Validate_Accuracy','Accuracy_Difference'])

# function to store accuracy for comparison purposes
def append_eval_df(model_type, train_accuracy, validate_accuracy):
    d = {'Model_Type': [model_type],'Train_Accuracy':[train_accuracy], 'Validate_Accuracy': [validate_accuracy], 'Accuracy_Difference': [train_accuracy - validate_accuracy]}
    d = pd.DataFrame(d)
    return eval_df.append(d, ignore_index = True)

In [11]:
# create baseline on mode (is_host == True)
train['baseline_pred'] = False
validate['baseline_pred'] = False
test['baseline_pred'] = False
# # print accuracy and classification report
# print('Train Accuracy: {:.2%}'.format(accuracy_score(train.is_host, train.baseline_pred)))
# print('---')
# print(classification_report(train.is_host, train.baseline_pred))

# print('Validate Accuracy: {:.2%}'.format(accuracy_score(validate.is_host, validate.baseline_pred)))
# print('---')
# print(classification_report(validate.is_host, validate.baseline_pred))
# append to eval df
eval_df = append_eval_df('baseline_pred', accuracy_score(train.is_host, train.baseline_pred),accuracy_score(validate.is_host, validate.baseline_pred))
eval_df

,Model_Type,Train_Accuracy,Validate_Accuracy,Accuracy_Difference
0,baseline_pred,0.503603,0.503611,-0.000008


In [12]:
# definee model; fit on X_ and y_train
lm = LogisticRegression().fit(X_train, y_train)
# add correspoonding predictions
train['predicted_X_train_just_features'] = lm.predict(X_train)
validate['predicted_X_validate_just_features'] = lm.predict(X_validate)
test['predicted_X_test_just_features'] = lm.predict(X_test)
# append t oeval df
eval_df = append_eval_df('Log_Reg_Just_Features', accuracy_score(train.actual, train.predicted_X_train_just_features),\
                         accuracy_score(validate.actual, validate.predicted_X_validate_just_features))
eval_df

ValueError: could not convert string to float: '2018-10-30'